## Generate static graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [ ]:
year = None
month = None
program = None

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

In [ ]:
import scaffolding 
from plots import *

In [ ]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_ei
%store -r dic_pur
%store -r dic_re

# convert a dictionary to a defaultdict
dic_pur = defaultdict(lambda: 'Other',dic_pur)
dic_re = defaultdict(lambda: 'Other',dic_re)

In [ ]:
tq = scaffolding.get_time_query(year, month)

In [ ]:
participant_ct_df = scaffolding.load_all_participant_trips(program, tq)

In [ ]:
labeled_ct = scaffolding.filter_labeled_trips(participant_ct_df)

In [ ]:
expanded_ct = scaffolding.expand_userinputs(labeled_ct)

In [ ]:
expanded_ct.shape

In [ ]:
expanded_ct = scaffolding.data_quality_check(expanded_ct)
expanded_ct.shape

In [ ]:
## Mapping new labels with dictionaries
expanded_ct['Trip_purpose']= expanded_ct['purpose_confirm'].map(dic_pur)
expanded_ct['Mode_confirm']= expanded_ct['mode_confirm'].map(dic_re)
expanded_ct['Replaced_mode']= expanded_ct['replaced_mode'].map(dic_re)

In [ ]:
dic_pur

In [ ]:
assert len(expanded_ct[(expanded_ct['Mode_confirm'] == 'Pilot ebike') & (expanded_ct["Replaced_mode"] == "Pilot ebike")]) == 0

In [ ]:
# Energy Impact Calculation
expanded_ct = scaffolding.energy_cal(expanded_ct,dic_ei)

In [ ]:
file_suffix = scaffolding.get_file_suffix(year, month, program)
quality_text = scaffolding.get_quality_text(participant_ct_df, expanded_ct)

# Distribution of Mode_confirm Attribute

In [ ]:
labels_mc = expanded_ct['Mode_confirm'].value_counts(dropna=True)[:9].keys().tolist()
values_mc = expanded_ct['Mode_confirm'].value_counts(dropna=True)[:9].tolist()
plot_title= "Number of trips for each mode (selected by users)\n%s" % quality_text
file_name= 'ntrips_mode_confirm_%s.png' % file_suffix
pie_chart_mode(plot_title,labels_mc,values_mc,file_name)

# Distribution of Replace_Mode Attribute

In [ ]:
labels_rm = expanded_ct['Replaced_mode'].value_counts(dropna=True)[:10].keys().tolist()
values_rm = expanded_ct['Replaced_mode'].value_counts(dropna=True)[:10].tolist()
plot_title="Number of trips for each replaced mode (selected by users)\n%s" % quality_text
file_name= 'ntrips_replaced_mode_%s.png' % file_suffix
pie_chart_mode(plot_title,labels_rm,values_rm,file_name)

### Note: The portion of pilot ebikes in the pie chart above is only when the mode_confirm is different that ebike. Example: mode_confirm was Walk, replaced_mode ebike. 
###           mode_confrim was Car, drove alone, replaced_mode ebike.

In [ ]:
expanded_ct.loc[expanded_ct['Replaced_mode'] == 'Pilot ebike'].Mode_confirm.unique()

# Distribution of Trip Purpose Attribute

In [ ]:
labels_tp = expanded_ct['Trip_purpose'].value_counts(dropna=True)[:10].keys().tolist()
values_tp = expanded_ct['Trip_purpose'].value_counts(dropna=True)[:10].tolist()
plot_title="Number of trips for each purposes (selected by users)\n%s" % quality_text
file_name= 'ntrips_purpose_%s.png' % file_suffix
pie_chart_purpose(plot_title,labels_tp,values_tp,file_name)

# Transport of Choice for Trips Under 10 Miles

In [ ]:
labels_d10 = expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True)[:10].keys().tolist()
values_d10 = expanded_ct.loc[(expanded_ct['distance_miles'] <= 10)].Mode_confirm.value_counts(dropna=True)[:10].tolist()
plot_title="Mode confirmations for trips under 10 Miles\n%s" % quality_text
file_name ='ntrips_under10miles_mode_confirm%s.png' % file_suffix
pie_chart_mode(plot_title,labels_d10,values_d10,file_name)

# Miles per transport mode selected (Mode_confirm)

In [ ]:
miles = expanded_ct.groupby('Mode_confirm').agg({'distance_miles': ['sum', 'count' , 'mean']})
miles.columns = ['Total (miles)', 'Count', 'Average (miles)']
miles = miles.reset_index()
miles =miles.sort_values(by=['Total (miles)'], ascending=False)

#data
miles_dict = dict(zip(miles['Mode_confirm'], miles['Total (miles)']))

labels_m = []
values_m = []

for x, y in miles_dict.items():
    labels_m.append(x)
    values_m.append(y)

labels_miles = labels_m[:5]
values_miles = values_m[:5]
plot_title="Miles for each mode (selected by users)\n%s" % quality_text
file_name ='miles_mode_confirm_%s.png' % file_suffix
pie_chart_mode(plot_title,labels_miles,values_miles,file_name)
print(miles)

# Average Miles per transport mode selected (Mode_confirm)

In [ ]:
data = miles.sort_values(by=['Average (miles)'], ascending=False)
x='Mode_confirm'
y='Average (miles)'

plot_title=" Average Miles for each mode (selected by users)\n%s" % quality_text
file_name ='average_miles_mode_confirm_%s.png' % file_suffix

barplot_mode(data,x,y,plot_title,file_name)

In [ ]:
data = miles.sort_values(by=['Average (miles)'], ascending=False)
x='Mode_confirm'
y='Average (miles)'
y2 = "Count"

plot_title=" Average Miles for each mode (selected by users)\n%s" % quality_text
file_name ='average_miles_mode_confirm_%s.png' % file_suffix

barplot_mode2(data,x,y,y2,plot_title,file_name)

# Number of Trips by Day¶

In [ ]:
fq_days = expanded_ct.groupby(['start_local_dt_day']).agg({'start_local_dt_day': ['sum', 'count']})
fq_days = fq_days.reset_index()
fq_days.columns = ['Day of the Month', 'Total', 'Number of Trips']
fq_days

In [ ]:
data = fq_days
x = 'Day of the Month'
y = 'Number of Trips'

plot_title="Number of trips by day\n%s" % quality_text
file_name ='number_of_trips_by_day_%s.png' % file_suffix

barplot_day(data,x,y,plot_title,file_name)